In [1]:
import pickle 


In [23]:
# open notebooks/paper_td2c/new_descriptors_only_ts_cmiknn_entropy/P10_N10_Nj2_n0.01_MB5.pkl 

import os
import pandas as pd

datas = []
for file in os.listdir("."):
    if file.endswith(".pkl"):
        with open(file, "rb") as f:
            data = pickle.load(f)
        datas.append(data)

new_descriptors = pd.concat(datas)

datas = []
for file in os.listdir("../descriptors_ts_cmiknn_entropy"):
    if file.endswith(".pkl"):
        if 'N5' in file:
            with open('../descriptors_ts_cmiknn_entropy/'+file, "rb") as f:
                data = pickle.load(f)
            datas.append(data)

old_descriptors = pd.concat(datas)

In [54]:
new_descriptors.max_neighborhood_size  .unique()

array([2])

In [35]:
old_descriptors.head(), old_descriptors.shape

(   process_id  graph_id  n_variables  max_neighborhood_size  noise_std  \
 0          13         0            5                      2       0.01   
 1          13         0            5                      2       0.01   
 2          13         0            5                      2       0.01   
 3          13         0            5                      2       0.01   
 4          13         0            5                      2       0.01   
 
    edge_source  edge_dest  is_causal  coeff_cause  coeff_eff  ...  \
 0           11          1          1    -0.000155   0.012058  ...   
 1           14          4          1    -0.004704  -0.007483  ...   
 2           14          1          1     0.104100   0.079396  ...   
 3            6          1          1     0.077293   0.075984  ...   
 4           12          0          1    -0.010915   0.002934  ...   
 
    mca_mca_cau_q1  mca_mca_cau_q2  mbe_mbe_eff_q0  mbe_mbe_eff_q1  \
 0       -0.031217       -0.031217             0.0      

In [55]:
old_descriptors[['process_id','graph_id','edge_source']].drop_duplicates()

,process_id,graph_id,edge_source
0,13,0,11
1,13,0,14
3,13,0,6
4,13,0,12
5,13,0,7
...,...,...,...
4913,4,39,21
4928,4,39,16
4929,4,39,26
4934,4,39,20


In [56]:
new_descriptors[['process_id','graph_id','edge_source']].drop_duplicates()

,process_id,graph_id,edge_source
0,13,0,11
1,13,0,14
3,13,0,6
4,13,0,12
5,13,0,7
...,...,...,...
4913,4,39,21
4928,4,39,16
4929,4,39,26
4934,4,39,20


In [70]:
merged_df = pd.merge(new_descriptors, old_descriptors, on=['process_id', 'graph_id', 'edge_source', 'edge_dest','n_variables', 'max_neighborhood_size', 'noise_std', 'is_causal'])


In [72]:
merged_df.columns

Index(['process_id', 'graph_id', 'n_variables', 'max_neighborhood_size',
       'noise_std', 'edge_source', 'edge_dest', 'is_causal', 'delta_mi',
       'delta_mi_after', 'coeff_cause', 'coeff_eff', 'HOC_3_1', 'HOC_1_2',
       'HOC_2_1', 'HOC_1_3', 'kurtosis_ca', 'kurtosis_ef', 'mca_mef_cau_q0',
       'mca_mef_cau_q1', 'mca_mef_cau_q2', 'mca_mef_eff_q0', 'mca_mef_eff_q1',
       'mca_mef_eff_q2', 'cau_m_eff_q0', 'cau_m_eff_q1', 'cau_m_eff_q2',
       'eff_m_cau_q0', 'eff_m_cau_q1', 'eff_m_cau_q2', 'm_cau_q0', 'm_cau_q1',
       'm_cau_q2', 'com_cau', 'cau_eff', 'eff_cau', 'eff_cau_mbeff',
       'cau_eff_mbcau', 'eff_cau_mbcau_plus_q0', 'eff_cau_mbcau_plus_q1',
       'eff_cau_mbcau_plus_q2', 'cau_eff_mbeff_plus_q0',
       'cau_eff_mbeff_plus_q1', 'cau_eff_mbeff_plus_q2', 'm_eff_q0',
       'm_eff_q1', 'm_eff_q2', 'mca_mca_cau_q0', 'mca_mca_cau_q1',
       'mca_mca_cau_q2', 'mbe_mbe_eff_q0', 'mbe_mbe_eff_q1', 'mbe_mbe_eff_q2',
       'n_samples', 'n_features', 'n_features/n_samples'

In [82]:
# lopo cross validation with group by process_id column classify on 'is_causal' column

from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, roc_auc_score

from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

from sklearn.preprocessing import StandardScaler

import numpy as np

X = merged_df[['coeff_cause', 'coeff_eff', 'HOC_3_1', 'HOC_1_2',
       'HOC_2_1', 'HOC_1_3', 'kurtosis_ca', 'kurtosis_ef', 'mca_mef_cau_q0',
       'mca_mef_cau_q1', 'mca_mef_cau_q2', 'mca_mef_eff_q0', 'mca_mef_eff_q1',
       'mca_mef_eff_q2', 'cau_m_eff_q0', 'cau_m_eff_q1', 'cau_m_eff_q2',
       'eff_m_cau_q0', 'eff_m_cau_q1', 'eff_m_cau_q2', 'm_cau_q0', 'm_cau_q1',
       'm_cau_q2', 'com_cau', 'cau_eff', 'eff_cau', 'eff_cau_mbeff',
       'cau_eff_mbcau', 'eff_cau_mbcau_plus_q0', 'eff_cau_mbcau_plus_q1',
       'eff_cau_mbcau_plus_q2', 'cau_eff_mbeff_plus_q0',
       'cau_eff_mbeff_plus_q1', 'cau_eff_mbeff_plus_q2', 'm_eff_q0',
       'm_eff_q1', 'm_eff_q2', 'mca_mca_cau_q0', 'mca_mca_cau_q1',
       'mca_mca_cau_q2', 'mbe_mbe_eff_q0', 'mbe_mbe_eff_q1', 'mbe_mbe_eff_q2',
       'n_samples', 'n_features', 'n_features/n_samples', 'skewness_ca',
       'skewness_ef']]
y = merged_df['is_causal']
groups = merged_df['process_id']

logo = LeaveOneGroupOut()

for train_index, test_index in logo.split(X, y, groups):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    clf = BalancedRandomForestClassifier(n_estimators=40, n_jobs=40, sampling_strategy='not majority', replacement=False, bootstrap=False)
    # clf = DecisionTreeClassifier()
    clf.fit(X_train, y_train)
    y_pred = clf.predict_proba(X_test)[:,1]
    # print(confusion_matrix(y_test, y_pred))
    print(roc_auc_score(y_test, y_pred))
    # print(classification_report(y_test, y_pred))
    print("")

0.8974603243543087

0.6900133851363974

0.6416903687277417

0.9568535925241849

0.6732635945967587

0.879157545184386

0.739050476351168

1.0

0.8832158384694218

0.9775115916298953

1.0

0.751882125637632

0.9729693820095606

1.0

0.9109428659516584

1.0

0.917330399701113

0.8912620903568546



In [14]:
0.9141603651957647

0.7257886835714588

0.8010488162025683

0.9425373668842846

0.715950620865117

0.9039997038823144

0.7749832026707

1.0

0.8856758797484434

0.9785041339170802

1.0

0.7420158289254353

0.9713477029325244

1.0

0.9059246529393461

1.0

0.9350675885362371

0.90096729859679


  Preparing metadata (setup.py) ... done
  Created wheel for pydotplus: filename=pydotplus-2.0.2-py3-none-any.whl size=24551 sha256=1b4be4d5a9c5fa0d7f342444b4da8a2db56391f85d2b2380ace63e263321fa1f
  Stored in directory: /home/gpaldino/.cache/pip/wheels/69/b2/67/08f0eef649af92df772c09f451558298e07fab1bc7cdf33db0
Successfully built pydotplus


In [15]:
# print decision tree
from sklearn.tree import export_graphviz
import pydotplus

dot_data = export_graphviz(clf, out_file=None,
                            feature_names=X.columns,
                            class_names=['False', 'True'],
                            filled=True, rounded=True,
                            special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data)
graph.write_png("decision_tree.png")

# print feature importance

True